# LSTM con Keras

Le reti **Long short-term memory (LSTM)** sono un'architettura di reti neurali ricorrenti che risolvono il problema della scomparsa del gradiente tra le diverse esecuzioni della rete.
<br>
Le LSTM  sono state da Sepp Hochreiter e Jurger Schmidhuber nel 1997 in [questa ricerca](https://www.bioinf.jku.at/publications/older/2604.pdf) ed in sostanza aggiungono uno stato alla rete (cell state) che viaggia in parallelo al segnale della rete e immagazzina le informazioni sequenziali.
<br><br>
In questo notebook utilizzeremo le LSTM per migliorare la rete neurale per la sentiment analysis addestrata sull'IMDB Movie Reviews dataset.

In [0]:
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense

## Scarichiamo il dataset
Utilizziamo Keras per caricare l'imdb dataset, limitandolo alle 10000 parole più comuni.

In [3]:
from keras.datasets import imdb 

num_words = 10000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

print("Numero di esempi nel train set: %d" % len(X_train))
print("Numero di esempi nel test set: %d" % len(X_test))

Numero di esempi nel train set: 25000
Numero di esempi nel test set: 25000


## Preprocessiamo i dati
Le recensioni all'interno del corpus di testo hanno ovviamente lunghezza differente, utilizziamo la funzione pad_sequences di keras per limitare le sequenze a 500 elementi (nel nostro caso limitare le frasi a 500 parole).
Se una sequenza ha meno di 500 esempi verranno aggiunti degli zeri alla fine.


In [4]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 500

X_train = pad_sequences(X_train, maxlen = maxlen)
X_test = pad_sequences(X_test, maxlen = maxlen)

X_train.shape

(25000, 500)

## Creiamo il modello
Costruiamo il nostro modello strutturandolo così:
1. Il primo strato eseguirà l'embedding creando 50 embedding vectors per ognuna delle 10.000 parole nel nostro dizionario.
2. Il secondo strato è lo strato ricorrente di tipo Long-short term memory.
3. Il terzo strato calcolerà l'ouput della rete, trattandosi di un problema di classifcazione binaria (recensione positiva/negativa) la funzione di attivazione sarà la sigmoide.

In [4]:
from keras.layers import Embedding, LSTM

model = Sequential()

model.add(Embedding(num_words, 50))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          500000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                10624     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 510,657
Trainable params: 510,657
Non-trainable params: 0
_________________________________________________________________


Compiliamo il modello ed eseguiamo l'addestramento per 5 epoche.

In [5]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=5)
model.evaluate(X_test, y_test)

Instructions for updating:
Use tf.cast instead.
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 37s 2ms/step - loss: 0.6412 - acc: 0.6766 - val_loss: 0.5538 - val_acc: 0.7566
Epoch 2/5
20000/20000 [==============================] - 34s 2ms/step - loss: 0.4363 - acc: 0.8274 - val_loss: 0.3925 - val_acc: 0.8382
Epoch 3/5
20000/20000 [==============================] - 34s 2ms/step - loss: 0.3244 - acc: 0.8763 - val_loss: 0.4741 - val_acc: 0.8024
Epoch 4/5
20000/20000 [==============================] - 34s 2ms/step - loss: 0.2708 - acc: 0.8995 - val_loss: 0.3731 - val_acc: 0.8464
Epoch 5/5
25000/25000 [==============================] - 119s 5ms/step


[0.32797487154483795, 0.87216]

Il risultato è nettamente migliore rispetto ad una semplice RNN, c'è un po' di overfitting, proviamo a correggerlo utilizzando il dropout.

## Dropout in una RNN
In una RNN possiamo utilizzare due differenti tipologie di dropout.
1. Dropout sull'input/output dello strato, esattamente come abbiamo già fatto con le altre architetture di reti neurali.
2. Dropout tra le esecuzioni ricorrenti della rete, questo permette di ridurre l'overfitting nelle features che contengono le informazioni sulla sequenza.

Per utilizzare il dropout sull'input di uno strato ricorrente, piuttosto che usare la classe Dropout, è consigliato sfruttare il parametro dropout delle classi SimpleRNN e LSTM.
Per utilizzare il dropout ricorrente possiamo invece utilizzare il parametro recurrent_dropout delle classi SimpleRNN e LSTM.

In [5]:
from keras.layers import Embedding, LSTM, Dropout

model = Sequential()

model.add(Embedding(num_words, 50))
model.add(LSTM(32, dropout=0.4, recurrent_dropout=0.2))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          500000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                10624     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 510,657
Trainable params: 510,657
Non-trainable params: 0
_________________________________________________________________


Compiliamo il modello ed eseguiamo l'addestramento per 10 epoche.

In [6]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=5)
model.evaluate(X_test, y_test)

Instructions for updating:
Use tf.cast instead.
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 41s 2ms/step - loss: 0.6590 - acc: 0.6369 - val_loss: 0.5685 - val_acc: 0.7596
Epoch 2/5
20000/20000 [==============================] - 40s 2ms/step - loss: 0.4999 - acc: 0.8003 - val_loss: 0.4337 - val_acc: 0.8184
Epoch 3/5
20000/20000 [==============================] - 40s 2ms/step - loss: 0.4029 - acc: 0.8409 - val_loss: 0.3740 - val_acc: 0.8418
Epoch 4/5
20000/20000 [==============================] - 40s 2ms/step - loss: 0.3614 - acc: 0.8567 - val_loss: 0.3482 - val_acc: 0.8534
Epoch 5/5
25000/25000 [==============================] - 139s 6ms/step


[0.4099230393791199, 0.81852]

Abbiamo ridotto l'overfitting nella nostra rete, adesso proviamo a migliorare il modello aggiungendo degli altri strati ricorrenti alla rete.

## Aggiungiamo altri strati LSTM
Possiamo aggiungere altri strati ricorrenti ad una rete nella solita maniera, utilizzando il metodo add della classe Sequential.
Tieni conto solo di una cosa, di default la classe LSTM esegue il flattening della sequenza per poterla dare come input ad uno strato denso, possiamo modificare tale comportamento impostando il parametro return_sequences a True.

In [12]:
from keras.layers import Embedding, LSTM, Dropout

model = Sequential()

model.add(Embedding(num_words, 100))
model.add(LSTM(32, dropout=0.5, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(32, dropout=0.5, recurrent_dropout=0.2))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
lstm_6 (LSTM)                (None, None, 32)          17024     
_________________________________________________________________
lstm_7 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 1,025,377
Trainable params: 1,025,377
Non-trainable params: 0
_________________________________________________________________


Compiliamo il modello ed eseguiamo l'addestramento per 10 epoche.

In [13]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=5)
model.evaluate(X_test, y_test)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 85s 4ms/step - loss: 0.6192 - acc: 0.6643 - val_loss: 0.5286 - val_acc: 0.7272
Epoch 2/5
20000/20000 [==============================] - 82s 4ms/step - loss: 0.4371 - acc: 0.8109 - val_loss: 0.4941 - val_acc: 0.7902
Epoch 3/5
20000/20000 [==============================] - 82s 4ms/step - loss: 0.3732 - acc: 0.8506 - val_loss: 0.3938 - val_acc: 0.8306
Epoch 4/5
20000/20000 [==============================] - 82s 4ms/step - loss: 0.3429 - acc: 0.8667 - val_loss: 0.3781 - val_acc: 0.8352
Epoch 5/5
25000/25000 [==============================] - 275s 11ms/step


[0.3764999796962738, 0.8376]

## CNN + LSTM

In [18]:
from keras.layers.convolutional import Conv1D, MaxPooling1D

model = Sequential()

model.add(Embedding(num_words, 50))
model.add(Conv1D(filters=32, kernel_size=3, padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(32, dropout=0.4))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 50)          500000    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 32)          4832      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 513,185
Trainable params: 513,185
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=5)
model.evaluate(X_test, y_test)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 23s 1ms/step - loss: 0.6243 - acc: 0.6450 - val_loss: 0.4240 - val_acc: 0.8290
Epoch 2/5
20000/20000 [==============================] - 20s 1ms/step - loss: 0.4186 - acc: 0.8144 - val_loss: 0.3573 - val_acc: 0.8614
Epoch 3/5
20000/20000 [==============================] - 20s 1ms/step - loss: 0.3249 - acc: 0.8629 - val_loss: 0.3128 - val_acc: 0.8726
Epoch 4/5
20000/20000 [==============================] - 20s 1ms/step - loss: 0.2735 - acc: 0.8909 - val_loss: 0.3690 - val_acc: 0.8562
Epoch 5/5
25000/25000 [==============================] - 66s 3ms/step


[0.2899127991676331, 0.87956]